In [1]:
# Cell 1 - imports & environment check
import os, re
from pathlib import Path
import pandas as pd
import spacy
import networkx as nx
import matplotlib.pyplot as plt

print("Working dir:", os.getcwd())

ModuleNotFoundError: No module named 'pandas'

In [2]:
# Cell 1 - imports & working dir check
import os, re
from pathlib import Path
import pandas as pd
import spacy
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

print("Working dir:", os.getcwd())
print("spacy version:", spacy.__version__)

ModuleNotFoundError: No module named 'pandas'

In [1]:
# Cell 1 - imports & environment check
import os, re
from pathlib import Path
import pandas as pd
import spacy
import networkx as nx
import matplotlib.pyplot as plt

print("Working dir:", os.getcwd())

C:\Users\Biswajit\anaconda3\envs\venv_20th_spacy\lib\site-packages\spacy\cli\info.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Working dir: C:\Users\Biswajit\20th-century-New


In [2]:
import spacy
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import re
from pathlib import Path

In [3]:
text_path = Path("20th_century_events.txt")
with open(text_path, "r", encoding="utf-8") as f:
    text = f.read()
    
print(text[:500])

FileNotFoundError: [Errno 2] No such file or directory: '20th_century_events.txt'

In [9]:
text_path = Path("20th_century_content.txt")
with open(text_path, "r", encoding="utf-8") as f:
    text = f.read()
    
print(text[:100])





Key events of the 20th century - Wikipedia












Main menu


move to sidebar


hide









In [10]:
# Cell 3: basic cleaning & show suspect characters
# Replace non-breaking space and collapse multi-whitespace
text_clean = text.replace("\xa0", " ")
text_clean = re.sub(r'\s+', ' ', text_clean)

# Detect unusual characters and a small sample of lines for manual check
samples = set(re.findall(r'[^\x00-\x7F]', text_clean))  # non-ASCII characters
print("Non-ASCII characters found (sample):", samples)
print("\nPreview after cleaning:\n", text_clean[:1000])

Non-ASCII characters found (sample): {'縄', 'ї', 'ع', 'ر', 'р', '’', 'é', '県', 'а', 'ي', '®', 'У', 'к', 'ã', 'ة', 'н', 'с', 'í', '沖', '—', 'ö', '–', 'ь', '°', 'ب', 'ل', 'ا', '／'}

Preview after cleaning:
  Key events of the 20th century - Wikipedia Main menu move to sidebar hide Navigation Main page Contents Current events Random article About Wikipedia Contact us Contribute Help Learn to edit Community portal Recent changes Upload file Special pages Search Search Appearance Appearance move to sidebar hide Text Small Standard Large This page always uses small font size Width Standard Wide The content is as wide as possible for your browser window. Color (beta) Automatic Light Dark This page is always in light mode. Donate Create account Log in Personal tools Donate Create account Log in Pages for logged out editors learn more Contributions Talk Toggle the table of contents Contents move to sidebar hide (Top) 1 Historic events in the 20th century Toggle Historic events in the 20th centur

In [11]:
# Cell 4: load nlp model and create doc
nlp = spacy.load("en_core_web_sm")
doc = nlp(text_clean)   # or use text if you didn't create text_clean
print("spaCy doc created. Number of tokens:", len(doc))

spaCy doc created. Number of tokens: 22866


In [12]:
# Cell 5: split into sentences and extract geographic/place entities
sentences = []
for sent in doc.sents:
    # extract entities with label GPE (countries/cities) or LOC
    ents = [ent.text for ent in sent.ents if ent.label_ in ("GPE","LOC")]
    sentences.append({"sentence": sent.text.strip(), "entities": ents})

df_sent = pd.DataFrame(sentences)
df_sent.head(10)

,sentence,entities
0,,[]
1,Key events of the 20th century - Wikipedia Mai...,[]
2,Contribute Help Learn to edit Community portal...,[]
3,Upload file Special pages Search Search Appear...,[]
4,Large This page always uses small font size,[]
5,Width Standard Wide The content is as wide as ...,[]
6,Color (beta) Automatic Light Dark,[]
7,This page is always in light mode.,[]
8,Donate Create account Log in Personal tools Do...,[]
9,Historic events in the 20th century Toggle His...,[]


In [13]:
# Cell 6: load country lookup (prefer CSV if you have)
# Try CSV first:
if Path("countries.csv").exists():
    df_countries = pd.read_csv("countries.csv")
    # try to find common column names
    col = None
    for c in ("country","Country","name","Name"):
        if c in df_countries.columns:
            col = c; break
    country_list = df_countries[col].astype(str).tolist() if col else df_countries.iloc[:,0].astype(str).tolist()
    print("Loaded countries from countries.csv")
else:
    # Fallback list — extend as needed
    country_list = ["United States","United Kingdom","France","Germany","Russia","China","Japan",
                    "Italy","Spain","India","Canada","Australia","Mexico","Brazil","Argentina","Soviet"]
    print("Using fallback country list (modify as needed)")

len(country_list)

Using fallback country list (modify as needed)


16

In [14]:
# Cell 7: filter entities to only the countries in your lookup list
country_set = set([c.lower() for c in country_list])

def keep_countries(ent_list):
    return [ent for ent in ent_list if ent.lower() in country_set]

df_sent["country_entities"] = df_sent["entities"].apply(keep_countries)
# keep only sentences that contain at least one of your countries
df_filtered = df_sent[df_sent["country_entities"].map(len) > 0].reset_index(drop=True)
df_filtered.head(10)

,sentence,entities,country_entities
0,After a period of diplomatic and military esca...,"[the British Empire, France, the Russian Empir...",[France]
1,"[ 1 ] [ 2 ] In 1917, Russia ended hostile acti...","[Russia, Tsar]",[Russia]
2,The Bolsheviks negotiated the Treaty of Brest-...,"[Bolsheviks, Germany, Russia]","[Germany, Russia]"
3,"In the treaty, Bolshevik Russia ceded the Balt...","[Germany, the Ottoman Empire]",[Germany]
4,[ 3 ] Although Germany shifted huge forces fro...,[Germany],[Germany]
5,"Germany, 1933 Fascism first appeared in Italy ...","[Germany, Italy]","[Germany, Italy]"
6,[ 29 ] When Adolf Hitler came to power in Germ...,"[Germany, Germany]","[Germany, Germany]"
7,The Nazi Party in Germany was dedicated to the...,"[Germany, Central and Eastern Europe]",[Germany]
8,[ 32 ] They could see nothing wrong with a str...,[Germany],[Germany]
9,"[ 33 ] Hitler began to put his plan in motion,...","[Austria, Austria, Germany]",[Germany]


In [15]:
# Cell 8: normalize common aliases (edit mapping as needed)
aliases = {"u.s.":"united states","u.s.a.":"united states","usa":"united states","ussr":"soviet"}
def normalize_entities(ent_list):
    out = []
    for ent in ent_list:
        e = ent.lower()
        e = e.replace(".", "")
        out.append(aliases.get(e, ent))  # replace if alias exists else original
    return out

# apply normalization (then re-filter with country_set)
df_filtered["country_entities"] = df_filtered["country_entities"].apply(normalize_entities)
df_filtered["country_entities"] = df_filtered["country_entities"].apply(lambda ents: [e for e in ents if e.lower() in country_set])
df_filtered = df_filtered[df_filtered["country_entities"].map(len) > 0].reset_index(drop=True)
df_filtered.head(10)

,sentence,entities,country_entities
0,After a period of diplomatic and military esca...,"[the British Empire, France, the Russian Empir...",[France]
1,"[ 1 ] [ 2 ] In 1917, Russia ended hostile acti...","[Russia, Tsar]",[Russia]
2,The Bolsheviks negotiated the Treaty of Brest-...,"[Bolsheviks, Germany, Russia]","[Germany, Russia]"
3,"In the treaty, Bolshevik Russia ceded the Balt...","[Germany, the Ottoman Empire]",[Germany]
4,[ 3 ] Although Germany shifted huge forces fro...,[Germany],[Germany]
5,"Germany, 1933 Fascism first appeared in Italy ...","[Germany, Italy]","[Germany, Italy]"
6,[ 29 ] When Adolf Hitler came to power in Germ...,"[Germany, Germany]","[Germany, Germany]"
7,The Nazi Party in Germany was dedicated to the...,"[Germany, Central and Eastern Europe]",[Germany]
8,[ 32 ] They could see nothing wrong with a str...,[Germany],[Germany]
9,"[ 33 ] Hitler began to put his plan in motion,...","[Austria, Austria, Germany]",[Germany]


In [16]:
# Cell 9A: relationships from same-sentence co-occurrence
pairs = []
for ents in df_filtered["country_entities"]:
    if len(ents) > 1:
        # create pairs (undirected): sort to keep order consistent
        for i in range(len(ents)):
            for j in range(i+1, len(ents)):
                a, b = sorted([ents[i], ents[j]])
                pairs.append((a, b))

df_rel = pd.DataFrame(pairs, columns=["source","target"])
df_rel["value"] = 1
df_rel = df_rel.groupby(["source","target"], as_index=False).sum().sort_values("value", ascending=False)
df_rel.head(20)

,source,target,value
12,Germany,Italy,4
8,France,Germany,4
4,China,India,3
5,China,Japan,3
13,Germany,Japan,3
19,Japan,Russia,2
18,Italy,Japan,2
17,India,Russia,1
16,India,Japan,1
15,India,India,1


In [17]:
# Cell 10: save the relationships for the network exercise
df_rel.to_csv("country_relationships.csv", index=False)
print("Saved country_relationships.csv — rows:", len(df_rel))

Saved country_relationships.csv — rows: 20
